## This notebook can be used to check for manually added events for which the phase (current, future, past) should be changed.

In [53]:
# Imports
import json
from hashlib import md5
from datetime import datetime

DB_FILE = '../docs/events_db.json'
EVENT_TAGS = ['exhibition', 'free']

def load_db():
    """Load the event database from a JSON file."""
    try:
        with open(DB_FILE, 'r') as file:
            return json.load(file)
    except FileNotFoundError:
        return {}

def save_db(db):
    """Save the event database to a JSON file."""
    with open(DB_FILE, 'w') as file:
        json.dump(db, file, indent=4, default=str)

def generate_event_hash(event_details):
    """Generate a hash of the event details to detect changes."""
    event_string = json.dumps(event_details, sort_keys=True, default=str)
    return md5(event_string.encode('utf-8')).hexdigest()

def generate_unique_identifier(event_details):
    """Generate a unique identifier for an event."""
    return f"{event_details['name']}-{event_details['venue']}"

def process_event(event_details):
    """Check if an event is new or has changed and update the database accordingly."""
    db = load_db()
    site_events = db.get(event_details['venue'], {})
    
    event_id = generate_unique_identifier(event_details)
    event_hash = generate_event_hash(event_details)

    if event_id not in site_events or site_events[event_id]['hash'] != event_hash:
        print(f"Updating event: {event_details['name']}")
        site_events[event_id] = {**event_details, 'hash': event_hash}
        db[event_details['venue']] = site_events
        save_db(db)
    else:
        print(f"No changes detected for event: {event_details['name']}")

In [62]:
def check_events():
    """
    Check all events for whether the 'past', 'current', or 'future' tags should be adjusted.
    """
    # Get today's date
    today = datetime.today().date()
    
    # Load events
    db = load_db()
    
    # Flatten the db into a list of event dictionaries
    events_list = []
    for category in db.values():
        for event in category.values():
            events_list.append(event)
            
    # Count events needing changes
    c = 0
                  
    # Check, for all events, whether the event has ended
    for event in events_list:
        # Try to convert the end date into a datetime object
        try:
            end_date = datetime.strptime(event['dates']['end'], '%Y-%m-%d').date()
        # If it can't be done, move on to the next event (this means the event passed the check)
        except ValueError:
            continue
        # Fail the test if the event is marked 'current' but the end date has passed
        if ('current' in event['tags']) and (end_date < today):
            c += 1
            print(f"{event['name']} @ {event['venue']} has changed from 'current' to 'past'. Phase and tags need to be updated.")

    # Check, for all events, whether any future events are now happening or have passed
    for event in events_list:
        # Try to convert the start date into a datetime object
        try:
            start_date = datetime.strptime(event['dates']['start'], '%Y-%m-%d').date()
            end_date = datetime.strptime(event['dates']['end'], '%Y-%m-%d').date()
        # If it can't be done, move on to the next event (this means the event passed the check)
        except ValueError:
            continue
        # Fail the test if the event is marked 'future' but the start date has passed and the end date hasn't
        if ('future' in event['tags']) and (start_date < today) and (end_date >= today):
            c += 1
            print(f"{event['name']} @ {event['venue']} has changed from 'future' to 'current'. Phase and tags need to be updated.")
        elif ('future' in event['tags']) and (end_date < today):
            c += 1
            print(f"{event['name']} @ {event['venue']} has changed from 'future' to 'past'. Phase and tags need to be updated.")
            
    print(f'{c} events need to be updated.')

In [63]:
check_events()

0 events need to be updated.
